In [1]:
import pandas as pd
import numpy as py
import operator
from ast import literal_eval

In [2]:
df_metadata= pd.read_csv("Resources/movies_metadata.csv", low_memory=False)
df_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
movies_cols=["id","title","production_countries","revenue"]
df_movies=df_metadata[movies_cols].copy()
df_movies.head()

,id,title,production_countries,revenue
0,862,Toy Story,"[{'iso_3166_1': 'US', 'name': 'United States o...",373554033.0
1,8844,Jumanji,"[{'iso_3166_1': 'US', 'name': 'United States o...",262797249.0
2,15602,Grumpier Old Men,"[{'iso_3166_1': 'US', 'name': 'United States o...",0.0
3,31357,Waiting to Exhale,"[{'iso_3166_1': 'US', 'name': 'United States o...",81452156.0
4,11862,Father of the Bride Part II,"[{'iso_3166_1': 'US', 'name': 'United States o...",76578911.0


In [4]:
renamed={"id":"ID", "title":"Title","production_countries": "Production Countries", "revenue":"Revenue in USD"}
df_movies = df_movies.rename(columns=renamed)
df_movies

,ID,Title,Production Countries,Revenue in USD
0,862,Toy Story,"[{'iso_3166_1': 'US', 'name': 'United States o...",373554033.0
1,8844,Jumanji,"[{'iso_3166_1': 'US', 'name': 'United States o...",262797249.0
2,15602,Grumpier Old Men,"[{'iso_3166_1': 'US', 'name': 'United States o...",0.0
3,31357,Waiting to Exhale,"[{'iso_3166_1': 'US', 'name': 'United States o...",81452156.0
4,11862,Father of the Bride Part II,"[{'iso_3166_1': 'US', 'name': 'United States o...",76578911.0
...,...,...,...,...
45461,439050,Subdue,"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",0.0
45462,111109,Century of Birthing,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",0.0
45463,67758,Betrayal,"[{'iso_3166_1': 'US', 'name': 'United States o...",0.0
45464,227506,Satan Triumphant,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",0.0


In [5]:
df_movies.dtypes

ID                       object
Title                    object
Production Countries     object
Revenue in USD          float64
dtype: object

In [6]:
df_movies["Revenue in USD"]=df_movies["Revenue in USD"].map("${:,.2f}".format)
df_movies.head()

,ID,Title,Production Countries,Revenue in USD
0,862,Toy Story,"[{'iso_3166_1': 'US', 'name': 'United States o...","$373,554,033.00"
1,8844,Jumanji,"[{'iso_3166_1': 'US', 'name': 'United States o...","$262,797,249.00"
2,15602,Grumpier Old Men,"[{'iso_3166_1': 'US', 'name': 'United States o...",$0.00
3,31357,Waiting to Exhale,"[{'iso_3166_1': 'US', 'name': 'United States o...","$81,452,156.00"
4,11862,Father of the Bride Part II,"[{'iso_3166_1': 'US', 'name': 'United States o...","$76,578,911.00"


In [7]:
df_movies.dropna(inplace=True)
df_movies.drop_duplicates(inplace=True)

In [8]:
converted_values = df_movies["Production Countries"].apply(literal_eval)
converted_values.head()

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
3    [{'iso_3166_1': 'US', 'name': 'United States o...
4    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: Production Countries, dtype: object

In [9]:
extracted_countries = []

for item in converted_values.values:
    if len(item) == 0:
        print('Item empty, skipping')
        extracted_countries.append('')
    else:
        print(item[0]['name'])
        extracted_countries.append(item[0]['name'])

United States of America
United States of America
United States of America
United States of America
United States of America
United States of America
Germany
United States of America
United States of America
United Kingdom
United States of America
France
United States of America
United States of America
France
France
United Kingdom
United States of America
United States of America
United States of America
United States of America
United States of America
France
United States of America
United States of America
Italy
United States of America
United Kingdom
France
China
United States of America
United States of America
France
Australia
France
United States of America
United States of America
United States of America
United States of America
South Africa
United Kingdom
United States of America
United States of America
United States of America
United States of America
United States of America
United States of America
United States of America
Canada
United States of America
Item empty, skip

In [10]:
df_movies["Production Countries"] = extracted_countries
df_movies.head()

,ID,Title,Production Countries,Revenue in USD
0,862,Toy Story,United States of America,"$373,554,033.00"
1,8844,Jumanji,United States of America,"$262,797,249.00"
2,15602,Grumpier Old Men,United States of America,$0.00
3,31357,Waiting to Exhale,United States of America,"$81,452,156.00"
4,11862,Father of the Bride Part II,United States of America,"$76,578,911.00"


In [11]:
df_movies.to_csv("Resources/movies_metadata_cora.csv")